In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import scipy
from PIL import Image
from scipy import ndimage
import gdal
import os
import geopandas as gpd
from skimage import io
from skimage.io import imread
%matplotlib inline

In [2]:
np.random.seed(1)

In [3]:
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans

C:\Users\Lenovo\anaconda3\envs\env_1\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


# reading the masked tif images and forming a dataset.

In [4]:
path = ["E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I1B1.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I1B2.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I1B3.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I1B4.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I1B5.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I2B1.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I2B2.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I2B3.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I2B4.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I2B5.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I3B1.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I3B2.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I3B3.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I3B4.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I3B5.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I4B1.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I4B2.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I4B3.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I4B4.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I4B5.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I5B1.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I5B2.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I5B3.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I5B4.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I5B5.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I6B1.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I6B2.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I6B3.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I6B4.tif",
       "E:\\Internship_Harvesting\\Dataset\\Bathinda_Cropland\\Masked_images\\I6B5.tif"]

In [5]:
I1B1 = gdal.Open(path[0])
I1B1

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002339FCB44E0> >

In [6]:
I1B1.RasterYSize, I1B1.RasterXSize, I1B1.RasterCount

(8963, 8298, 1)

In [7]:
I1B1_array = I1B1.GetRasterBand(1).ReadAsArray()
I1B1_array

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [8]:
np.count_nonzero(np.isnan(I1B1_array)), np.count_nonzero(~np.isnan(I1B1_array))

(40574909, 33800065)

# creating an array which only consists the non-nan values indices

In [9]:
# I1B1_nan_index = np.argwhere(np.isnan(I1B1))

I1B1_non_nan_index = np.argwhere(~np.isnan(I1B1_array))

In [10]:
I1B1_non_nan_index

array([[   0, 5738],
       [   0, 5739],
       [   0, 5740],
       ...,
       [8962, 5788],
       [8962, 5789],
       [8962, 5790]], dtype=int64)

In [11]:
I1B1_array[0, 5738], I1B1_array[0, 5739], I1B1_array[8962, 5789], I1B1_array[8962, 5790]

(1249.0, 1242.0, 1248.0, 1237.0)

In [12]:
I1B1_non_nan_index.shape

(33800065, 2)

# using these indices, creating a numpy array for each band of image

### IMAGE 1 BAND 1

In [13]:
I1B1_array_non_nan = np.zeros((1, 33800065))

In [14]:
sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I1B1_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I1B1_array_non_nan = np.concatenate((sample, I1B1_array_non_nan), axis = 0)

In [15]:
print(I1B1_array_non_nan.shape)
print("\n**************\n")
I1B1_array_non_nan

(2, 33800065)

**************



array([[1249., 1242., 1209., ..., 1251., 1248., 1237.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [16]:
I1B1_array_non_nan = I1B1_array_non_nan[:1, :]
I1B1_array_non_nan.shape

(1, 33800065)

In [17]:
I1B1_array_non_nan

array([[1249., 1242., 1209., ..., 1251., 1248., 1237.]])

### IMAGE 1 BAND 2

In [18]:
I1B2 = gdal.Open(path[1])

print(I1B2.RasterYSize, I1B2.RasterXSize, I1B2.RasterCount)
print("\n***************************\n")

I1B2_array = I1B2.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I1B2_array)), np.count_nonzero(~np.isnan(I1B2_array)))

8963 8298 1

***************************

40574909 33800065


In [19]:
I1B2_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I1B2_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I1B2_array_non_nan = np.concatenate((sample, I1B2_array_non_nan), axis = 0)

print(I1B2_array_non_nan.shape)
print("\n**************\n")
print(I1B2_array_non_nan)
print("\n**************\n")

I1B2_array_non_nan = I1B2_array_non_nan[:1, :]
print(I1B2_array_non_nan.shape)
print("\n**************\n")

print(np.count_nonzero(np.isnan(I1B2_array_non_nan)), np.count_nonzero(~np.isnan(I1B2_array_non_nan)))
print("\n**************\n")
I1B2_array_non_nan

(2, 33800065)

**************

[[1200. 1182. 1184. ... 1139. 1130. 1144.]
 [   0.    0.    0. ...    0.    0.    0.]]

**************

(1, 33800065)

**************

0 33800065

**************



array([[1200., 1182., 1184., ..., 1139., 1130., 1144.]])

In [20]:
I1B2_array[0, 5738], I1B2_array[0, 5739], I1B2_array[8962, 5789], I1B2_array[8962, 5790]

(1200.0, 1182.0, 1130.0, 1144.0)

### correlation between band 1 and band 2  of image 1

In [21]:
np.corrcoef(I1B1_array_non_nan, I1B2_array_non_nan)

array([[1.        , 0.84473379],
       [0.84473379, 1.        ]])

### IMAGE 1 BAND 3

In [22]:
I1B3 = gdal.Open(path[2])

print(I1B3.RasterYSize, I1B3.RasterXSize, I1B3.RasterCount)
print("\n***************************\n")

I1B3_array = I1B3.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I1B3_array)), np.count_nonzero(~np.isnan(I1B3_array)))

8963 8298 1

***************************

40574909 33800065


In [23]:
I1B3_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I1B3_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I1B3_array_non_nan = np.concatenate((sample, I1B3_array_non_nan), axis = 0)

print(I1B3_array_non_nan.shape)
print("\n**************\n")
print(I1B3_array_non_nan)
print("\n**************\n")

I1B3_array_non_nan = I1B3_array_non_nan[:1, :]
print(I1B3_array_non_nan.shape)
print("\n**************\n")

print(np.count_nonzero(np.isnan(I1B3_array_non_nan)), np.count_nonzero(~np.isnan(I1B3_array_non_nan)))
print("\n**************\n")
I1B3_array_non_nan

(2, 33800065)

**************

[[978.  968.  954.  ... 983.5 960.  949. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]

**************

(1, 33800065)

**************

0 33800065

**************



array([[978. , 968. , 954. , ..., 983.5, 960. , 949. ]])

In [24]:
I1B3_array[0, 5738], I1B3_array[0, 5739], I1B3_array[8962, 5789], I1B3_array[8962, 5790]

(978.0, 968.0, 960.0, 949.0)

### IMAGE 1 BAND 4

In [25]:
I1B4 = gdal.Open(path[3])

print(I1B4.RasterYSize, I1B4.RasterXSize, I1B4.RasterCount)
print("***************************")

I1B4_array = I1B4.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I1B4_array)), np.count_nonzero(~np.isnan(I1B4_array)))

8963 8298 1
***************************
40574909 33800065


In [26]:
I1B4_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I1B4_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I1B4_array_non_nan = np.concatenate((sample, I1B4_array_non_nan), axis = 0)

I1B4_array_non_nan = I1B4_array_non_nan[:1, :]

print(np.count_nonzero(np.isnan(I1B4_array_non_nan)), np.count_nonzero(~np.isnan(I1B4_array_non_nan)))
print("**************")
I1B4_array_non_nan

0 33800065
**************


array([[1405., 1405., 1404., ..., 1338., 1338., 1342.]])

In [27]:
I1B4_array[0, 5738], I1B4_array[0, 5739], I1B4_array[8962, 5789], I1B4_array[8962, 5790]

(1405.0, 1405.0, 1338.0, 1342.0)

### IMAGE 1 BAND 5

In [28]:
I1B5 = gdal.Open(path[4])

print(I1B5.RasterYSize, I1B5.RasterXSize, I1B5.RasterCount)
print("***************************")

I1B5_array = I1B5.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I1B5_array)), np.count_nonzero(~np.isnan(I1B5_array)))

8963 8298 1
***************************
40574909 33800065


In [29]:
I1B5_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I1B5_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I1B5_array_non_nan = np.concatenate((sample, I1B5_array_non_nan), axis = 0)

I1B5_array_non_nan = I1B5_array_non_nan[:1, :]

print(np.count_nonzero(np.isnan(I1B5_array_non_nan)), np.count_nonzero(~np.isnan(I1B5_array_non_nan)))
print("**************")
I1B5_array_non_nan

0 33800065
**************


array([[2622. , 2757. , 2787. , ..., 2800. , 2755.5, 2731. ]])

In [30]:
I1B5_array[0, 5738], I1B5_array[0, 5739], I1B5_array[8962, 5789], I1B5_array[8962, 5790]

(2622.0, 2757.0, 2755.5, 2731.0)

### creating a numpy with combining all the arrays of each band in one.

In [31]:
IMAGE1_non_nan_array = np.concatenate((I1B1_array_non_nan, I1B2_array_non_nan, I1B3_array_non_nan,
                                      I1B4_array_non_nan, I1B5_array_non_nan), axis = 0)

In [32]:
IMAGE1_non_nan_array.shape

(5, 33800065)

In [33]:
IMAGE1_non_nan_array

array([[1249. , 1242. , 1209. , ..., 1251. , 1248. , 1237. ],
       [1200. , 1182. , 1184. , ..., 1139. , 1130. , 1144. ],
       [ 978. ,  968. ,  954. , ...,  983.5,  960. ,  949. ],
       [1405. , 1405. , 1404. , ..., 1338. , 1338. , 1342. ],
       [2622. , 2757. , 2787. , ..., 2800. , 2755.5, 2731. ]])

### correlation matrix of all bands for image 1

In [34]:
np.corrcoef(IMAGE1_non_nan_array)

array([[ 1.        ,  0.84473379,  0.8491875 ,  0.53175924, -0.19103249],
       [ 0.84473379,  1.        ,  0.87377881,  0.82273054,  0.07250003],
       [ 0.8491875 ,  0.87377881,  1.        ,  0.70709481, -0.26158604],
       [ 0.53175924,  0.82273054,  0.70709481,  1.        ,  0.23400101],
       [-0.19103249,  0.07250003, -0.26158604,  0.23400101,  1.        ]])

#### Observation ::: band 2 is showing high collinearity with band1, band3, and band4
#### Also band5 is not showing any significant collinearity with any other bands.

### IMAGE 2 BAND 1

In [36]:
I2B1 = gdal.Open(path[5])

print(I2B1.RasterYSize, I2B1.RasterXSize, I2B1.RasterCount)
print("***************************")

I2B1_array = I2B1.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I2B1_array)), np.count_nonzero(~np.isnan(I2B1_array)))

8963 8298 1

***************************

40574909 33800065


In [37]:
I2B1_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I2B1_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I2B1_array_non_nan = np.concatenate((sample, I2B1_array_non_nan), axis = 0)

I2B1_array_non_nan = I2B1_array_non_nan[:1, :]

print(np.count_nonzero(np.isnan(I2B1_array_non_nan)), np.count_nonzero(~np.isnan(I2B1_array_non_nan)))
print("**************")
I2B1_array_non_nan

0 33800065
**************


array([[1447., 1467., 1492., ..., 1253., 1236., 1277.]])

In [38]:
I2B1_array[0, 5738], I2B1_array[0, 5739], I2B1_array[8962, 5789], I2B1_array[8962, 5790]

(1447.0, 1467.0, 1236.0, 1277.0)

### IMAGE 2 BAND 2

In [40]:
I2B2 = gdal.Open(path[6])

print(I2B2.RasterYSize, I2B2.RasterXSize, I2B2.RasterCount)
print("***************************")

I2B2_array = I2B2.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I2B2_array)), np.count_nonzero(~np.isnan(I2B2_array)))

8963 8298 1
***************************
40574909 33800065


In [41]:
I2B2_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I2B2_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I2B2_array_non_nan = np.concatenate((sample, I2B2_array_non_nan), axis = 0)

I2B2_array_non_nan = I2B2_array_non_nan[:1, :]

print(np.count_nonzero(np.isnan(I2B2_array_non_nan)), np.count_nonzero(~np.isnan(I2B2_array_non_nan)))
print("**************")
I2B2_array_non_nan

0 33800065
**************


array([[1322.5, 1367. , 1398.5, ..., 1120. , 1125. , 1121. ]])

In [42]:
I2B2_array[0, 5738], I2B2_array[0, 5739], I2B2_array[8962, 5789], I2B2_array[8962, 5790]

(1322.5, 1367.0, 1125.0, 1121.0)

### IMAGE 2 BAND 3

In [44]:
I2B3 = gdal.Open(path[7])

print(I2B3.RasterYSize, I2B3.RasterXSize, I2B3.RasterCount)
print("***************************")

I2B3_array = I2B3.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I2B3_array)), np.count_nonzero(~np.isnan(I2B3_array)))

8963 8298 1
***************************
40574909 33800065


In [45]:
I2B3_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I2B3_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I2B3_array_non_nan = np.concatenate((sample, I2B3_array_non_nan), axis = 0)

I2B3_array_non_nan = I2B3_array_non_nan[:1, :]

print(np.count_nonzero(np.isnan(I2B3_array_non_nan)), np.count_nonzero(~np.isnan(I2B3_array_non_nan)))
print("**************")
I2B3_array_non_nan

0 33800065
**************


array([[1431. , 1535. , 1592.5, ..., 1029. , 1005. , 1019. ]])

In [46]:
I2B3_array[0, 5738], I2B3_array[0, 5739], I2B3_array[8962, 5789], I2B3_array[8962, 5790]

(1431.0, 1535.0, 1005.0, 1019.0)

### IMAGE 2 BAND 4

In [48]:
I2B4 = gdal.Open(path[8])

print(I2B4.RasterYSize, I2B4.RasterXSize, I2B4.RasterCount)
print("***************************")

I2B4_array = I2B4.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I2B4_array)), np.count_nonzero(~np.isnan(I2B4_array)))

8963 8298 1
***************************
40574909 33800065


In [49]:
I2B4_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I2B4_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I2B4_array_non_nan = np.concatenate((sample, I2B4_array_non_nan), axis = 0)

I2B4_array_non_nan = I2B4_array_non_nan[:1, :]

print(np.count_nonzero(np.isnan(I2B4_array_non_nan)), np.count_nonzero(~np.isnan(I2B4_array_non_nan)))
print("**************")
I2B4_array_non_nan

0 33800065
**************


array([[1626.5, 1626.5, 1761. , ..., 1240. , 1240. , 1242. ]])

In [50]:
I2B4_array[0, 5738], I2B4_array[0, 5739], I2B4_array[8962, 5789], I2B4_array[8962, 5790]

(1626.5, 1626.5, 1240.0, 1242.0)

### IMAGE 2 BAND 5

In [52]:
I2B5 = gdal.Open(path[9])

print(I2B5.RasterYSize, I2B5.RasterXSize, I2B5.RasterCount)
print("***************************")

I2B5_array = I2B5.GetRasterBand(1).ReadAsArray()

print(np.count_nonzero(np.isnan(I2B5_array)), np.count_nonzero(~np.isnan(I2B5_array)))

8963 8298 1
***************************
40574909 33800065


In [53]:
I2B5_array_non_nan = np.zeros((1, 33800065))

sample = []

for i in range(I1B1_non_nan_index.shape[0]):
    sample.append(I2B5_array[I1B1_non_nan_index[i,0], I1B1_non_nan_index[i,1]])
    
sample = np.array(sample)

sample = sample.reshape(1, 33800065)

I2B5_array_non_nan = np.concatenate((sample, I2B5_array_non_nan), axis = 0)

I2B5_array_non_nan = I2B5_array_non_nan[:1, :]

print(np.count_nonzero(np.isnan(I2B5_array_non_nan)), np.count_nonzero(~np.isnan(I2B5_array_non_nan)))
print("**************")
I2B5_array_non_nan

0 33800065
**************


array([[1831.5, 1872. , 1967.5, ..., 2475. , 2450. , 2447. ]])

In [54]:
I2B5_array[0, 5738], I2B5_array[0, 5739], I2B5_array[8962, 5789], I2B5_array[8962, 5790]

(1831.5, 1872.0, 2450.0, 2447.0)

### concatenating all arrays of image 2 in one

In [55]:
IMAGE2_non_nan_array = np.concatenate((I2B1_array_non_nan, I2B2_array_non_nan, I2B3_array_non_nan,
                                      I2B4_array_non_nan, I2B5_array_non_nan), axis = 0)

### correlation of all bands of image 2

In [58]:
np.corrcoef(IMAGE2_non_nan_array)

array([[1.        , 0.92952375, 0.79378606, 0.68593218, 0.1192663 ],
       [0.92952375, 1.        , 0.91431244, 0.82980288, 0.22165909],
       [0.79378606, 0.91431244, 1.        , 0.90258291, 0.16519888],
       [0.68593218, 0.82980288, 0.90258291, 1.        , 0.44213763],
       [0.1192663 , 0.22165909, 0.16519888, 0.44213763, 1.        ]])

### Observation ::: it is clear that band1, band2, band3 and band4 are higly correlated with one another.
### band5 does not show much collinearity with other bands

# In order to reduce the feature we would randomly sample 1% of non-nan values and merge them having all 30 features, also we will merge the output(prediction) from k-means with 30 features and will find correlation matrix

In [59]:
np.random.seed(1)

In [60]:
a = np.arange(I1B1_non_nan_index.shape[0])

a_rand = np.random.choice(a, 338000)

In [61]:
rand_index = I1B1_non_nan_index[a_rand, :]
rand_index

array([[2643, 3574],
       [ 767, 4965],
       [3732, 6427],
       ...,
       [3434, 3074],
       [5842, 4036],
       [6873, 4799]], dtype=int64)

In [63]:
rand_index.shape

(338000, 2)

In [64]:
test_x_sample = np.zeros((1, 338000))

In [65]:
for p in path:
    print(p)
    
    image = gdal.Open(p)
    
    image_array = image.GetRasterBand(1).ReadAsArray()
    print(image_array.shape)
    sample = []
    
    for i in range(rand_index.shape[0]):
        sample.append(image_array[rand_index[i,0], rand_index[i,1]])
        
    sample = np.array(sample)
    
    sample = sample.reshape(1, 338000)
        
    test_x_sample = np.concatenate((sample, test_x_sample), axis = 0)
    
    print(test_x_sample.shape)
    
    print("***************")
    

E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B1.tif
(8963, 8298)
(2, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B2.tif
(8963, 8298)
(3, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B3.tif
(8963, 8298)
(4, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B4.tif
(8963, 8298)
(5, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B5.tif
(8963, 8298)
(6, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B1.tif
(8963, 8298)
(7, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B2.tif
(8963, 8298)
(8, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B3.tif
(8963, 8298)
(9, 338000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B4.tif
(8963,

In [66]:
test_x_sample = test_x_sample[:30, :]
test_x_sample = test_x_sample.T
test_x_sample.shape

(338000, 30)

In [67]:
kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, n_init=10, random_state=0)
test_y_sample = kmeans.fit_predict(test_x_sample)

In [68]:
np.unique(test_y_sample)

array([0, 1])

In [70]:
test_y_sample = test_y_sample.T
test_x_sample = test_x_sample.T

In [71]:
test_y_sample.shape, test_x_sample.shape

((338000,), (30, 338000))

In [72]:
test_y_sample = test_y_sample.reshape(1, 338000)
test_y_sample.shape

(1, 338000)

In [73]:
test_x_y_sample = np.concatenate((test_x_sample, test_y_sample), axis = 0)
test_x_y_sample.shape

(31, 338000)

In [74]:
corr_mat_sample = np.corrcoef(test_x_y_sample)
corr_mat_sample.shape

(31, 31)

In [76]:
corr_mat_sample_df = pd.DataFrame(data = corr_mat_sample)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.000000,-0.455118,-0.580124,-0.443235,-0.497289,0.493684,-0.403528,-0.467211,-0.330251,-0.337118,...,0.190948,0.136024,0.122808,0.065186,0.373229,-0.007675,-0.265050,-0.169739,-0.264249,-0.038321
1,-0.455118,1.000000,0.916599,0.927931,0.878158,-0.399705,0.856344,0.808258,0.761455,0.698897,...,0.037783,0.033131,0.101049,0.117518,-0.299864,0.230072,0.574325,0.492662,0.600995,-0.035206
2,-0.580124,0.916599,1.000000,0.965013,0.955924,-0.464752,0.823747,0.878110,0.811862,0.758427,...,0.024034,0.039953,0.103246,0.125152,-0.323323,0.223069,0.618023,0.530604,0.646819,-0.030843
3,-0.443235,0.927931,0.965013,1.000000,0.966011,-0.445237,0.836009,0.874683,0.847655,0.794100,...,0.025673,0.025272,0.104607,0.122878,-0.293488,0.205299,0.600490,0.530118,0.662744,-0.057057
4,-0.497289,0.878158,0.955924,0.966011,1.000000,-0.510134,0.829382,0.891161,0.859423,0.845383,...,0.003528,-0.001423,0.081839,0.107560,-0.310494,0.183024,0.594893,0.520711,0.697668,-0.100016
5,0.493684,-0.399705,-0.464752,-0.445237,-0.510134,1.000000,-0.491375,-0.640733,-0.472276,-0.552160,...,0.296577,0.430898,0.331304,0.272398,0.380050,0.304873,-0.098735,0.037104,-0.284996,0.143222
6,-0.403528,0.856344,0.823747,0.836009,0.829382,-0.491375,1.000000,0.904312,0.907433,0.831543,...,0.012670,-0.059586,0.034875,0.048645,-0.249331,0.155082,0.506382,0.424138,0.595169,-0.095141
7,-0.467211,0.808258,0.878110,0.874683,0.891161,-0.640733,0.904312,1.000000,0.932560,0.908205,...,-0.033577,-0.086929,0.011952,0.038497,-0.307913,0.101866,0.533962,0.435920,0.644599,-0.123913
8,-0.330251,0.761455,0.811862,0.847655,0.859423,-0.472276,0.907433,0.932560,1.000000,0.943989,...,0.018078,-0.031165,0.075607,0.093697,-0.219488,0.171264,0.543836,0.482647,0.665830,-0.151268
9,-0.337118,0.698897,0.758427,0.794100,0.845383,-0.552160,0.831543,0.908205,0.943989,1.000000,...,-0.032871,-0.085874,0.028694,0.060005,-0.234161,0.119455,0.501147,0.442224,0.684361,-0.223539


In [77]:
corr_mat_sample_df.to_csv("E:\\Internship_Harvesting\\Unsupervised_correlation_matrix\\corr_mat.csv")

## doing the same process for another sample but now with 10%

In [13]:
a = np.arange(I1B1_non_nan_index.shape[0])

a_rand = np.random.choice(a, 3380000)

In [14]:
rand_index = I1B1_non_nan_index[a_rand, :]
rand_index

array([[2643, 3574],
       [ 767, 4965],
       [3732, 6427],
       ...,
       [3759, 5550],
       [2416, 2802],
       [6447, 2610]], dtype=int64)

In [15]:
rand_index.shape

(3380000, 2)

In [16]:
test_x_sample = np.zeros((1, 3380000))

In [17]:
for p in path:
    print(p)
    
    image = gdal.Open(p)
    
    image_array = image.GetRasterBand(1).ReadAsArray()
    print(image_array.shape)
    sample = []
    
    for i in range(rand_index.shape[0]):
        sample.append(image_array[rand_index[i,0], rand_index[i,1]])
        
    sample = np.array(sample)
    
    sample = sample.reshape(1, 3380000)
        
    test_x_sample = np.concatenate((sample, test_x_sample), axis = 0)
    
    print(test_x_sample.shape)
    
    print("***************")
    

E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B1.tif
(8963, 8298)
(2, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B2.tif
(8963, 8298)
(3, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B3.tif
(8963, 8298)
(4, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B4.tif
(8963, 8298)
(5, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I1B5.tif
(8963, 8298)
(6, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B1.tif
(8963, 8298)
(7, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B2.tif
(8963, 8298)
(8, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B3.tif
(8963, 8298)
(9, 3380000)
***************
E:\Internship_Harvesting\Dataset\Bathinda_Cropland\Masked_images\I2B4.ti

In [18]:
test_x_sample = test_x_sample[:30, :]
test_x_sample = test_x_sample.T
test_x_sample.shape

(3380000, 30)

In [19]:
kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, n_init=10, random_state=0)
test_y_sample = kmeans.fit_predict(test_x_sample)

In [20]:
np.unique(test_y_sample)

array([0, 1])

In [21]:
test_y_sample = test_y_sample.T
test_x_sample = test_x_sample.T

In [22]:
test_y_sample.shape, test_x_sample.shape

((3380000,), (30, 3380000))

In [24]:
test_y_sample = test_y_sample.reshape(1, 3380000)
test_y_sample.shape

(1, 3380000)

In [25]:
test_x_y_sample = np.concatenate((test_x_sample, test_y_sample), axis = 0)
test_x_y_sample.shape

(31, 3380000)

In [26]:
test_x_y_sample

array([[3.1460e+03, 3.2075e+03, 3.5260e+03, ..., 2.5835e+03, 2.6545e+03,
        3.5235e+03],
       [1.0870e+03, 9.0100e+02, 9.1600e+02, ..., 1.8830e+03, 9.7350e+02,
        8.9200e+02],
       [7.6950e+02, 6.1050e+02, 5.9000e+02, ..., 1.7395e+03, 6.7500e+02,
        5.7550e+02],
       ...,
       [1.2340e+03, 1.1230e+03, 1.1740e+03, ..., 1.7260e+03, 1.3650e+03,
        1.3050e+03],
       [1.2490e+03, 1.1330e+03, 1.2120e+03, ..., 1.6620e+03, 1.2980e+03,
        1.2630e+03],
       [0.0000e+00, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [27]:
# feature normalizing

for i in range(test_x_y_sample.shape[0]):
    averg = np.mean(test_x_y_sample[i, :])
    std_dev = np.std(test_x_y_sample[i, :])
    test_x_y_sample[i, :] = (test_x_y_sample[i, :] - averg) / std_dev

In [28]:
test_x_y_sample

array([[-2.48263675e-01, -1.07242106e-01,  6.23089106e-01, ...,
        -1.53809509e+00, -1.37528971e+00,  6.17356522e-01],
       [ 5.80737755e-01, -5.67356207e-01, -4.74767984e-01, ...,
         5.49408611e+00, -1.19846464e-01, -6.22909141e-01],
       [ 4.14644634e-01, -4.11012564e-01, -5.17465222e-01, ...,
         5.45167282e+00, -7.60761536e-02, -5.92761004e-01],
       ...,
       [-8.92756485e-02, -1.20449977e+00, -6.92099499e-01, ...,
         4.85387993e+00,  1.22688976e+00,  6.24065908e-01],
       [-1.63676071e-01, -1.56068708e+00, -6.09274408e-01, ...,
         4.81016483e+00,  4.26440647e-01,  4.92870563e-03],
       [-4.83985963e-01, -4.83985963e-01,  2.06617563e+00, ...,
        -4.83985963e-01, -4.83985963e-01, -4.83985963e-01]])

In [29]:
corr_mat_sample = np.corrcoef(test_x_y_sample)
corr_mat_sample.shape

(31, 31)

In [30]:
corr_mat_sample_df = pd.DataFrame(data = corr_mat_sample)

In [33]:
corr_mat_sample_df.to_csv("E:\\Internship_Harvesting\\Unsupervised_correlation_matrix\\corr_mat_10percent.csv")